In [1]:
### Import de bibliotecas para manipulação dos dados
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer

In [6]:
#Caminho e leitura do Dataframe
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

#Análise das 10 primeiras linhas 
df.head(5)

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5


In [4]:
# Leitura do CSV
df_dicionario_dados = pd.read_csv('../data/external/dicionario_de_dados_laranjas.csv')

#Análise das 10 primeiras linhas 
df_dicionario_dados.head(10)

,Variável,Tradução,Descrição,Espectro de opções,Tipo,Subtipo
0,Size (cm),Tamanho,Diâmetro da laranja em centímetros,(6.0 -- 10.0),Quantitativo,Contínua
1,Weight (g),Peso,Peso da laranja em gramas,(100.0 -- 300.0),Quantitativo,Contínua
2,Brix (Sweetness),Doçura,Nível de açúcar da laranja,(5.5 -- 16.0),Quantitativo,Contínua
3,pH (Acidity),Acidez,Nível de acidez da laranja,(2.8 -- 4.4),Quantitativo,Contínua
4,Softness (1-5),Maciez,Nível de maciez da laranja em uma escala de 1 a 5,(1-5),Quantitativo,Discreto
5,HarvestTime (days),Tempo de colheita,Número de dias desde o plantio até a colheita,(4 -- 25),Quantitativo,Discreto
6,Ripeness (1-5),Maturação,Nível de maturação da laranja em uma escala de...,(1-5),Quantitativo,Discreto
7,Color,Cor,Cor da casca da laranja,"Orange,Deep Orange,Light Orange,Orange-Red,Yel...",Qualitativo,Nominal
8,Blemishes (Y/N),Manchas,Presença de manchas na laranja,"Y (Minor),Y (Sunburn),Y (Mold Spot),Y (Bruise)...",Qualitativo,Nominal
9,Quality (1-5),Qualidade,Qualidade geral da laranja,(1-5),Quantitativo,Discreto


In [ ]:
#Checagem de nomes de coluna para facilitar elaboração do código 
#print(df_dicionario_dados.columns)

In [7]:
#Organização da busca pelo tipo de variável

variavel_alvo = 'Quality (1-5)'

# Renomeando colunas (opcional)
df_dicionario_dados = df_dicionario_dados.rename(columns={'Variável': 'Variavel', 'Subtipo': 'Subtipo'})


variavel_nominal = (
    df_dicionario_dados
    .query("Subtipo == 'Nominal' and Variavel != @variavel_alvo")
    .Variavel
    .to_list()
)
print(f"Variáveis qualitativo: nominal")
print(variavel_nominal)
print(" ")

variavel_quanti_continua = (
    df_dicionario_dados
    .query("Subtipo == 'Contínua' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Contínua")
print(variavel_quanti_continua)
print(" ")

variavel_quanti_discreta = (
    df_dicionario_dados
    .query("Subtipo == 'Discreto' and Variavel != @variavel_alvo")
    .Variavel #Nome da coluna que tem as variáveis
    .to_list() #Método que lista
)
print(f"Variável quantitativo: Discreto")
print(variavel_quanti_discreta)
print(" ")

#x = df.drop(columns=[variavel_alvo])
#y = df[variavel_alvo]

Variáveis qualitativo: nominal
['Color', 'Blemishes (Y/N)', 'Variety']
 
Variável quantitativo: Contínua
['Size (cm)', 'Weight (g)', 'Brix (Sweetness)', 'pH (Acidity)']
 
Variável quantitativo: Discreto
['Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)']
 


## Pré-processamento 

In [ ]:
processamento_nominal = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento para dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False)), # codificação de variáveis
])

processamento_continuos = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')), #Tratamento de dados faltantes com substituição pela MÉDIA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])
processamento_discreto = Pipeline(steps=[
    ("missing", KNNImputer()), #Tratamento de dados faltantes com substituição pela MEDIANA
    ("normalization", StandardScaler()), #Normalização de variáveis
     
])


## Combinar transformações e aplicar

In [ ]:
pre_processador = ColumnTransformer([
    ("Contínua", processamento_continuos, variavel_quanti_continua),
    ("Discreto", processamento_discreto, variavel_quanti_discreta),
    ("Nominal", processamento_nominal, variavel_nominal)
])

df_preprocessed = pre_processador.fit_transform(df)

df_preprocessed

## K-fold (Validação Cruzada)

In [ ]:
#k-fold
# Create a Pipeline
model = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),  # Optional scaler for numerical features
    ('regressor', LinearRegression())
])

# Perform cross-validation
scores = cross_val_score(model, df, y, cv=10, scoring='neg_mean_squared_error')

print(scores)

mean_score = np.mean(scores)
std_score = np.std(scores)

print("Média:", mean_score)
print("Desvio Padrão:", std_score)

#### Como é possivel observar, foram usados 10 folds para esse modelo. Os resultados da validação cruzada indicaram um valor de -0.36 de MSE, sugerindo que o modelo é capaz de fazer previsões com uma precisão razoável. O Desvio padrão 0.14 do MSE demonstra a robustez do modelo em diferentes subconjuntos dos dados.